<a href="https://colab.research.google.com/github/tylerjamesryan/Simulation/blob/master/par_variance_gamma_v_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import libraries

if (!require(lme4)) {install.packages("lme4"); library(lme4)}
if (!require(dplyr)) {install.packages("lme4"); library(dplyr)}

Loading required package: lme4

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lme4’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘nloptr’, ‘RcppEigen’


Loading required package: Matrix

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [27]:


f <- function(N = 10, J = 1000, s1 = .5) {
  require(lme4)
  require(dplyr)
  J <- 100
  N <- 10
  id <- rep(1:J, each = N)
  x <- rnorm(N * J, 0, 1)
  b00 <- 1
  b0j <- rnorm(J, 0, 1)
  b1 <- .5
  s1 <- s1
  s00 <- 1
  s0j <- rnorm(J, 0, 1)
  s2 <- (s00 + s0j[id] + s1*x)^2
  s <- sqrt(s2)
  mu <- b00 + b0j[id] + b1*x
  Y <- rnorm(J*N, mu, s)
  Ycen <- Y-mean(Y)
  Ycen2 <- Ycen^2
  ycen <- Y - mean(Y) - ave(Y, id, FUN = mean) 
  ycen2 <- ycen^2
  df <- data.frame(id = id, x = x, Y = Y, s = s, Y2 = Ycen2, y2 = ycen2)

  mod_s1 <- glmer(Y2 ~ 1 + x + (1|id), family = Gamma(link = "log"),
                  data = df, control = glmerControl("bobyqa"))
  df2 <- df %>%
    group_by(id) %>%
    summarize(rmse = sqrt(mean(Y2)),
              x = mean(x))
  b_agg <- exp(coef(lm(I(log(rmse)) ~ 1 + x, df2)))
  gam <- coef(lm(I(cbind(1, df2$x) %*% b_agg) ~ 0 + df2$rmse))
  b_agg_corrected <- gam * b_agg

    c("b0" = s00, 
      "b1" = s1, 
      "glmm" = 2*summary(mod_s1)$coefficients[,c(1)],
      "lm" = b_agg,
      "lm_c" = b_agg_corrected)
}



In [28]:

sims <- vector("list", 3)
s1s <- c(.1, .4, .6)[1:3]
for (i in 1:3) {
  s_i <- replicate(2, f(N = 30, J = 10000, s1 = s1s[i]))
  sims[[i]] <- as.data.frame(t(s_i))
  print(i)
}

[1] 1
[1] 2


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0243714 (tol = 0.002, component 1)”


[1] 3


In [29]:
do.call(rbind, sims)

b0,b1,glmm.(Intercept),glmm.x,lm.(Intercept),lm.x,lm_c.(Intercept),lm_c.x
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.1,1.740954,0.2024861,1.541041,0.9931317,1.052414,0.6782340
1,0.1,1.767124,0.2236325,1.556584,1.2178449,1.185965,0.9278790
1,0.4,1.704592,0.4188250,1.536438,1.1476753,1.112741,0.8311858
1,0.4,1.561586,0.3971372,1.470297,0.8973469,1.013322,0.6184474
1,0.6,1.639555,0.4963124,1.528297,0.9155674,1.207942,0.7236497
1,0.6,1.879742,0.4245883,1.600465,1.0341241,1.145864,0.7403884
